In [ ]:
import os
import sys
import json
from src.ts_main import run_ts, parse_input_dict

In [ ]:
TS_BASE_DIR = ".."

# Example 1

In [ ]:
quinazoline_json = """{
"reagent_file_list": [
        "TS_BASE_DIR/data/aminobenzoic_100.smi",
        "TS_BASE_DIR/data/primary_amines_100.smi",
        "TS_BASE_DIR/data/carboxylic_acids_100.smi"
    ],
    "reaction_smarts": "N[c:4][c:3]C(O)=O.[#6:1][NH2].[#6:2]C(=O)[OH]>>[C:2]c1n[c:4][c:3]c(=O)n1[C:1]",
    "num_warmup_trials": 10,
    "num_ts_iterations": 5000,
    "evaluator_class_name": "FPEvaluator",
    "evaluator_arg": {"query_smiles" : "COc1cccc2c(=O)n([C@@H]3C[C@@H](O)[C@@H](O)C3)c([C@@H](N)CNC(N)=O)nc12"},
    "ts_mode": "maximize",
    "log_filename": "ts_logs.txt",
    "results_filename": "ts_results.csv"
}""".replace("TS_BASE_DIR", TS_BASE_DIR)

In [ ]:
quinazoline_dict = json.loads(quinazoline_json)
parse_input_dict(quinazoline_dict)

In [ ]:
run_ts(quinazoline_dict, inline_log=True, save_output=False)

# Example 2

In [2]:
import os
import sys
import json
from src.ts_main import run_ts, parse_input_dict

In [3]:
TS_BASE_DIR = ".."

In [4]:
custom_json = """{
"reagent_file_list": [
        "TS_BASE_DIR/data/data_ric/ald.smi",
        "TS_BASE_DIR/data/data_ric/ammines.smi"
    ],
    "reaction_smarts": "TS_BASE_DIR/data/data_ric/reaction.rxn",
    "num_warmup_trials": 10,
    "num_ts_iterations": 5000,
    "evaluator_class_name": "FPEvaluator",
    "evaluator_arg": {"query_smiles" : "OC(=O)c1ccc2nc(CN3CCC(CC3)c3cccc(OCc4ccc(cc4F)C#N)n3)n(C[C@@H]3CCO3)c2c1"},
    "ts_mode": "maximize",
    "log_filename": "ts_logs.txt",
    "results_filename": "ts_results.csv"
}""".replace("TS_BASE_DIR", TS_BASE_DIR)

In [5]:
custom_dict = json.loads(custom_json)
parse_input_dict(custom_dict)

In [6]:
run_ts(custom_dict, inline_log=True, save_output=False)

2024-10-14:13:34:25,293 INFO     /home/hitesit/Python_Packages/Machine_Learning/Thomson_Samling/TS/src/thompson_sampling.py:94 2.61e+04 possible products


Warmup 1 of 2:   0%|          | 0/179 [00:00<?, ?it/s]

Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on

Warmup 2 of 2:   0%|          | 0/146 [00:00<?, ?it/s]

Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on

...

2024-10-14:13:34:38,605 INFO     /home/hitesit/Python_Packages/Machine_Learning/Thomson_Samling/TS/src/thompson_sampling.py:247 Iteration: 4900 max score: 0.230769 smiles: C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)C#N)CC4=CN5C(=N4)C=CC(=C5NCC6=CC=C(C=C6)C(=O)O)C7=NN=NN7 ald_76_ammine_27


Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on

2024-10-14:13:34:38,767 INFO     /home/hitesit/Python_Packages/Machine_Learning/Thomson_Samling/TS/src/ts_main.py:83 8183 evaluations | 31.312% of total


Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch
Going on branch


,score,SMILES,Name
0,0.230769,C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)C#N)CC4=CN5C(=N...,ald_76_ammine_27
1,0.178295,C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)C(=O)O)CC4=CN5C...,ald_106_ammine_27
2,0.220472,C1CO[C@@H]1CNC2=C(C=CC3=NC(=CN32)CN4CCC(CC4)N5...,ald_76_ammine_142
3,0.213740,C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)C#N)CC4=CN5C(=N...,ald_76_ammine_66
4,0.207692,C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)C#N)CC4=CN5C(=N...,ald_76_ammine_36
...,...,...,...
4995,0.152672,CC1=CC=C(C=C1)CNC2=C(C=CC3=NC(=CN32)CN4CCC(CC4...,ald_105_ammine_118
4996,0.150376,CC(C)(C)CC(C)(C)NC1=C(C=CC2=NC(=CN21)CN3CCC(CC...,ald_121_ammine_80
4997,0.137405,C1CN(CCC1N2C=NC=C2C3=CC=C(C=C3)Br)CC4=CN5C(=N4...,ald_102_ammine_105
4998,0.161765,C1CN(CCC1N2C=NC=C2C3=CC4=C(C=C3)N=CN4)CC5=CN6C...,ald_9_ammine_12
